##Keras

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import numpy as np

In [0]:
from scipy.spatial.distance import cdist

def eval_MAP_K(query_set, search_set, query_label, search_set_label, K):
    query_class_arr = query_set
    search_set_class_arr = search_set

    query_label_arr = query_label
    query_label_arr.astype(int)
    search_set_label_arr = search_set_label
    search_set_label_arr.astype(int)

    num_query_sample = query_label_arr.shape[0]

    dist_q_search_class = cdist(query_class_arr,search_set_class_arr,metric='euclidean')

    AP=0
    for query_count in range(0,num_query_sample):

        actual_label = query_label_arr[query_count]
        value=dist_q_search_class[query_count,:]
        sorted_value_idx=value.argsort()
        predicted_K_label = search_set_label_arr[sorted_value_idx[0:K]]
        L=np.sum((np.in1d(predicted_K_label,actual_label)).astype(int))

        P=0
        for r in range(0,K):
            rtr_label_r = predicted_K_label[:(r+1)]
            l=np.sum((np.in1d(rtr_label_r,actual_label)).astype(int))
            precision=l/(r+1)
            if actual_label==predicted_K_label[r]:
                delta=1
            else:
                delta=0
            P=P+(precision*delta)
        if L!=0:
            AP=AP+((1/L)*P)
    map_K=AP/num_query_sample
    return map_K

In [0]:
sketchdata = np.load('/content/gdrive/My Drive/feature_vectors_final/siamese/skdat.npy')      #sketch feature data
sketchlabels = np.load('/content/gdrive/My Drive/feature_vectors_final/siamese/sklab.npy')    #sketch labels
imagedata = np.load('/content/gdrive/My Drive/feature_vectors_final/siamese/imgdat.npy')      #image feature data
imagelabels = np.load('/content/gdrive/My Drive/feature_vectors_final/siamese/imglab.npy')    #image labels

In [0]:
from sklearn import preprocessing
enc = preprocessing.OneHotEncoder(sparse = False)

In [6]:
imagelabels = enc.fit_transform(imagelabels.reshape(-1,1))

sketchlabels = enc.fit_transform(sketchlabels.reshape(-1,1))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [7]:
#-------------------------------encoder-----------------------------------------------------------------

input_img = Input(shape=(4096,))

#layer1_img = Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.1))(input_img)

# "encoded" is the encoded representation of the input

encoded_img = Dense(125,activation='relu')(input_img)


#--------------------------------decoder-------------------------------------------------------------------


#layer2_img = Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.1))(encoded_img)

# "decoded" is the lossy reconstruction of the input

decoded_img = Dense(4096,activation='relu')(encoded_img)



# this model maps an input to its reconstruction
autoencoder_img = Model(input_img, [decoded_img,encoded_img])

W0723 05:48:19.073744 140192443914112 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
encoder_img = Model(input_img, encoded_img)

In [9]:
autoencoder_img.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4096)]            0         
_________________________________________________________________
dense (Dense)                (None, 125)               512125    
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              516096    
Total params: 1,028,221
Trainable params: 1,028,221
Non-trainable params: 0
_________________________________________________________________


In [10]:
encoder_img.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4096)]            0         
_________________________________________________________________
dense (Dense)                (None, 125)               512125    
Total params: 512,125
Trainable params: 512,125
Non-trainable params: 0
_________________________________________________________________


In [0]:
#---------------------------------encoder-----------------------------------------------------

input_skt = Input(shape=(4096,))

# "encoded" is the encoded representation of the input
#layer1_skt = Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.1))(input_skt)

encoded_skt = Dense(125,activation = 'relu')(input_skt)


#---------------------------------decoder------------------------------------------------------


#layer2_skt = Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.1))(encoded_skt)


# "decoded" is the lossy reconstruction of the input
decoded_skt = Dense(4096, activation='relu')(encoded_skt)

# this model maps an input to its reconstruction
autoencoder_skt = Model(input_skt, [decoded_skt,encoded_skt])

In [0]:
encoder_skt = Model(input_skt, encoded_skt)

In [13]:
autoencoder_skt.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 4096)]            0         
_________________________________________________________________
dense_2 (Dense)              (None, 125)               512125    
_________________________________________________________________
dense_3 (Dense)              (None, 4096)              516096    
Total params: 1,028,221
Trainable params: 1,028,221
Non-trainable params: 0
_________________________________________________________________


In [14]:
encoder_skt.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 4096)]            0         
_________________________________________________________________
dense_2 (Dense)              (None, 125)               512125    
Total params: 512,125
Trainable params: 512,125
Non-trainable params: 0
_________________________________________________________________


###Parallel model

In [0]:
parllel = Model([input_skt,input_img],[decoded_skt,encoded_skt,decoded_img,encoded_img])

In [16]:
parllel.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 4096)]       0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 4096)]       0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 125)          512125      input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 125)          512125      input_1[0][0]                    
____________________________________________________________________________________________

In [0]:
from tensorflow.keras.optimizers import Adam 
parllel.compile(optimizer=Adam(lr=1e-4),loss=['mean_squared_error','mean_squared_error','mean_squared_error','mean_squared_error'])

In [0]:
def calc():
    encoded_sketch = encoder_skt.predict(sketch_set)
    encoded_image = encoder_img.predict(image_set)
    return eval_MAP_K(encoded_sketch, encoded_image, sketch_lab, image_lab, 100)

In [0]:
from tensorflow import keras
class callbacker(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print("  MAP = {} ".format(calc()))
        return calc()

In [0]:
sketch_set = np.load('/content/gdrive/My Drive/feature_vectors_final/sketches_test_collated/final.npy')
sketch_lab = np.load('/content/gdrive/My Drive/feature_vectors_final/sketches_test_collated/labels.npy')
image_set = np.load('/content/gdrive/My Drive/image features/xtest.npy')
image_lab = np.load('/content/gdrive/My Drive/image features/ytest.npy')

In [21]:
from sklearn.preprocessing import LabelEncoder

image_lab = LabelEncoder().fit_transform(image_lab)
sketch_lab = LabelEncoder().fit_transform(sketch_lab)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
callbacked=callbacker()
parllel.fit([sketchdata,imagedata],[sketchdata, sketchlabels, imagedata, imagelabels], shuffle=True, epochs=150, batch_size=128, verbose=1, callbacks=[callbacked])

Epoch 1/150
55831/55831 [==============================] - 74s 1ms/sample - loss: 0.8214 - dense_3_loss: 0.0064 - dense_2_loss: 0.0251 - dense_1_loss: 0.6976 - dense_loss: 0.0924
Epoch 2/150
55831/55831 [==============================] - 75s 1ms/sample - loss: 0.8206 - dense_3_loss: 0.0063 - dense_2_loss: 0.0250 - dense_1_loss: 0.6971 - dense_loss: 0.0922
Epoch 3/150
55831/55831 [==============================] - 76s 1ms/sample - loss: 0.8199 - dense_3_loss: 0.0064 - dense_2_loss: 0.0248 - dense_1_loss: 0.6966 - dense_loss: 0.0920
Epoch 4/150
55831/55831 [==============================] - 76s 1ms/sample - loss: 0.8192 - dense_3_loss: 0.0063 - dense_2_loss: 0.0247 - dense_1_loss: 0.6965 - dense_loss: 0.0917
Epoch 5/150
55831/55831 [==============================] - 75s 1ms/sample - loss: 0.8179 - dense_3_loss: 0.0062 - dense_2_loss: 0.0246 - dense_1_loss: 0.6957 - dense_loss: 0.0915
Epoch 6/150
55831/55831 [==============================] - 75s 1ms/sample - loss: 0.8171 - dense_3_loss: 

In [0]:
encoded_sketch = encoder_skt.predict(sketch_set)
encoded_image = encoder_img.predict(image_set)

In [27]:
eval_MAP_K(encoded_sketch, encoded_image, sketch_lab, image_lab, 100)

0.024317567041931904

In [28]:
encoded_sketch[1]

array([0.7219325 , 0.        , 0.70069826, 0.        , 0.66803557,
       0.62685585, 0.7574718 , 0.        , 0.        , 0.        ,
       0.72757393, 0.        , 0.7200924 , 0.        , 0.8587819 ,
       0.        , 0.69630945, 0.        , 0.7292609 , 0.        ,
       0.75707823, 0.        , 0.7240025 , 0.        , 0.6801532 ,
       0.66487867, 0.7053518 , 0.        , 0.71848947, 0.8259611 ,
       0.6554199 , 0.        , 0.        , 0.74681693, 0.7405156 ,
       0.        , 0.78049046, 0.        , 0.        , 0.8069914 ,
       0.        , 0.        , 0.9219955 , 0.78738207, 0.72992784,
       0.6431669 , 0.7010893 , 0.        , 0.        , 0.        ,
       0.        , 0.68200934, 0.67716044, 0.        , 0.        ,
       0.        , 0.7657473 , 0.        , 0.        , 0.        ,
       0.        , 0.713563  , 0.74309635, 0.        , 0.        ,
       0.77570343, 0.7482069 , 0.71594256, 0.        , 0.        ,
       0.        , 0.        , 0.8648931 , 0.79049516, 0.     

In [29]:
encoded_image[1]

array([0.11268716, 0.44332576, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.6609948 , 0.        , 0.20870043,
       0.        , 0.        , 0.        , 0.        , 0.34981123,
       0.13411716, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.86618704, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.96028

###Training seperately

In [0]:
from tensorflow.keras.optimizers import Adam 
autoencoder_skt.compile(optimizer=Adam(lr=1e-4),loss=['mean_squared_error','mean_squared_error'])

In [0]:
autoencoder_skt.fit(sketchdata,[sketchdata, sketchlabels], shuffle=True, epochs=100, batch_size=128, verbose=1)

Epoch 1/100
55831/55831 [==============================] - 7s 117us/sample - loss: 0.0555 - dense_7_loss: 0.0465 - dense_5_loss: 0.0090
Epoch 2/100
55831/55831 [==============================] - 7s 119us/sample - loss: 0.0548 - dense_7_loss: 0.0458 - dense_5_loss: 0.0090
Epoch 3/100
55831/55831 [==============================] - 7s 122us/sample - loss: 0.0547 - dense_7_loss: 0.0457 - dense_5_loss: 0.0090
Epoch 4/100
55831/55831 [==============================] - 7s 124us/sample - loss: 0.0548 - dense_7_loss: 0.0458 - dense_5_loss: 0.0090
Epoch 5/100
55831/55831 [==============================] - 7s 119us/sample - loss: 0.0551 - dense_7_loss: 0.0461 - dense_5_loss: 0.0090
Epoch 6/100
55831/55831 [==============================] - 6s 116us/sample - loss: 0.0545 - dense_7_loss: 0.0455 - dense_5_loss: 0.0090
Epoch 7/100
55831/55831 [==============================] - 6s 115us/sample - loss: 0.0543 - dense_7_loss: 0.0450 - dense_5_loss: 0.0093
Epoch 8/100
55831/55831 [=======================

In [0]:
from tensorflow.keras.optimizers import Adam 
autoencoder_img.compile(optimizer=Adam(lr=1e-4),loss=['mean_squared_error','mean_squared_error'])

In [0]:
autoencoder_img.fit(imagedata,[imagedata, imagelabels], shuffle=True, epochs=100, batch_size=128, verbose=1)

Epoch 1/100
55831/55831 [==============================] - 7s 126us/sample - loss: 0.3983 - dense_3_loss: 0.3654 - dense_1_loss: 0.0329
Epoch 2/100
55831/55831 [==============================] - 7s 126us/sample - loss: 0.3975 - dense_3_loss: 0.3647 - dense_1_loss: 0.0328
Epoch 3/100
55831/55831 [==============================] - 7s 128us/sample - loss: 0.3963 - dense_3_loss: 0.3636 - dense_1_loss: 0.0327
Epoch 4/100
55831/55831 [==============================] - 7s 127us/sample - loss: 0.3954 - dense_3_loss: 0.3627 - dense_1_loss: 0.0327
Epoch 5/100
55831/55831 [==============================] - 7s 129us/sample - loss: 0.3942 - dense_3_loss: 0.3616 - dense_1_loss: 0.0326
Epoch 6/100
55831/55831 [==============================] - 7s 130us/sample - loss: 0.3931 - dense_3_loss: 0.3606 - dense_1_loss: 0.0325
Epoch 7/100
55831/55831 [==============================] - 7s 131us/sample - loss: 0.3923 - dense_3_loss: 0.3598 - dense_1_loss: 0.0325
Epoch 8/100
55831/55831 [=======================

In [0]:
sketch_set = np.load('/content/gdrive/My Drive/feature_vectors_final/sketches_test_collated/final.npy')
sketch_lab = np.load('/content/gdrive/My Drive/feature_vectors_final/sketches_test_collated/labels.npy')
image_set = np.load('/content/gdrive/My Drive/image features/xtest.npy')
image_lab = np.load('/content/gdrive/My Drive/image features/ytest.npy')

In [0]:
from sklearn.preprocessing import LabelEncoder

image_lab = LabelEncoder().fit_transform(image_lab)
sketch_lab = LabelEncoder().fit_transform(sketch_lab)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
encoded_sketch = encoder_skt.predict(sketch_set)
encoded_image = encoder_img.predict(image_set)

In [0]:
eval_MAP_K(encoded_sketch, encoded_image, sketch_lab, image_lab, 100)

0.024014342163513377